# MovieLens数据集实现协同过滤
## 协同过滤概述

协同过滤（Collaborative Filtering）推荐算法是最经典、最常用的推荐算法。

协同过滤主要包括基于用户相似度的推荐(UserCF)和基于物品相似度推荐(ItemCF)。但是在生活实际中，一个用户一般来说只与少部分物品和少部分用户有关联，所以共现矩阵是稀疏的。此时我们要用已有的部分稀疏数据来预测那些空白的物品和数据之间的评分关系，找到最高评分的物品推荐给用户。

- **基于用户(user-based)的协同过滤**
    - **相似的用户可能喜欢相同物品**。比如用户A和用户B有共同的兴趣或者行为，现在要给用户B推荐物品，就可以把用户A喜欢的物品，并且用户B没看过的物品，推荐给用户B。
- **基于物品(item-based)的协同过滤**
    - **相似的物品可能被同个用户喜欢**。世界杯期间沃尔玛尿布和啤酒的故事。在世界杯期间，奶爸要喝啤酒看球，又要带娃，啤酒和尿布同时被奶爸所需要，也就是相似商品，可以放在一起销售。本来啤酒和尿布在生活中，并没什么相关性，是通过用户的行为来判断相关性的。
- **基于模型(model based)的协同过滤**
    - 使用矩阵分解模型来学习用户和物品的协同过滤信息。一般这种协同过滤模型有：SVD，SVD++等。

## 实现协同过滤，需要几个步骤：

1. 收集用户偏好(评分，浏览记录，购买记录等)
2. 找到相似的用户或物品
3. 计算并排序

## 评估指标

比如，你给用户推10个物品，但是只推了5个是用户想要的(正确的)，准确率就是<font size=4 width="50%" height="50%">$\frac{5}{10}$</font>，而这个用户实际想要的有20个，但是只命中了5个，所以召回率就是<font size=4 width="50%" height="50%">$\frac{5}{20}$<font>。

R(u)是用户u的推荐列表(根据用户在训练集上的行为给用户预测的物品集合)，T(u)是用户在测试集上的行为列表

### 准确率
准确率：推荐对的物品数量占召回总数量比例
<div align=center><font size=5 width="50%" height="50%">$Presicion=\frac{\sum_{u \in U}|R(u)| \cap |T(u)|}{\sum_{u \in U}|R(u)|}$</font></div>

### 召回率
召回率：推荐对的物品数量占总物品数量的比例
<div align=center><font size=5 width="50%" height="50%">$Recall=\frac{\sum_{u \in U}|R(u)| \cap |T(u)|}{\sum_{u \in U}|T(u)|}$</font></div>

# 用户协同
## 目标
1. 找到目标用户A相似的K个用户
2. 将相似的K个用户曾经看过的物品，并且目标用户A没有买过或浏览过的物品推荐给目标用户A

## 用户相似度计算
假设给定用户u和用户v，N(u)是用户u曾经看过的物品集合，N(v)是用户v曾经看过的用户集合
![image.png](用户-物品行为记录.png)

jaccard计算用户u和用户v相似度
<div align=center><font size=5 width="50%" height="50%">$W_{uv}=\frac{|N(u)\cap N(v)|}{|N(u)\cup N(v)|}$</font></div>

余弦计算相似度
<div align=center><font size=5 width="50%" height="50%">$W_{uv}=\frac{|N(u)\cap N(v)|}{\sqrt|N(u)||N(v)|}$</font></div>

伪代码：
```python
def UserSimilarity(train):
    W = dict()
    for u in train.keys():
        for v in train.keys():
            if u == v:
                continue
            W[u][v] = len(train[u] & train[v])
            W[u][v] /= math.sqrt(len(train[u]) * len(train[v]) * 1.0)
    return W
```

<font size=4>**思考：这么计算有没有什么问题？**</font>

1. 每次计算，都要遍历一遍用户表，时间复杂度是$O(n^2)$
2. 用户和用户计算相似度过程中，假如用户之间没有共同看过的物品的，也就是上面的交集为空，这种情况也会被保存

<font size=4>**解决**</font>

既然有很多用户没有打分，并且这个计算复杂度这么高，我们可不可以换一种思路，上面那是用户-物品表，换成建立物品-用户的倒排表。

![image.png](物品用户倒排表.png)

<font size=4>**物品-用户倒排表好处？**</font>

1. 用户没有过行为的，直接被过滤了，最终我们得到的都是有行为的表，计算上减少了，时间复杂度也降低了
2. 将计算的用户u和用户v的相似度矩阵存储起来，大大减少计算量

## 给用户推荐和他最相似的K个用户喜欢的物品
用户对物品i感兴趣程度
<div align=center><font size=5 width="50%" height="50%">$p(u,i)=\sum_{v \in S(u,K) \cap N(i)}W_{uv}r_{vi}$</font></div>
$S(u,K)$ 包含和用户$u$兴趣最接近的$K$个用户，$N(i)$是对物品$i$有过行为的用户集合，$W_{uv}$是用户$u$和用户$v$的兴趣相似度，$r_{vi}$是用户$v$对物品的感兴趣程度

![image.png](用户对物品i的相似度.png)


## 用户协同代码实现

In [1]:
import os
import math
import random
from pandas import DataFrame
class UserBasedCF:
    def __init__(self, path):
        self.train = {} #用户-物品的评分表 训练集
        self.test = {} #用户-物品的评分表 测试集
        self.generate_dataset(path)

    def loadfile(self, path):
        with open(path, 'r', encoding='utf-8') as fp:
            for i, line in enumerate(fp):
                yield line.strip('\r\n')

    
    def generate_dataset(self, path, pivot=0.7):
        #读取文件，并生成用户-物品的评分表和测试集
        i = 0
        for line in self.loadfile(path):
            user, movie, rating, _ = line.split('::')
            if i <= 10:
                print('{},{},{},{}'.format(user, movie, rating, _))
            i += 1
            if random.random() < pivot:
                self.train.setdefault(user, {})
                self.train[user][movie] = int(rating)
            else:
                self.test.setdefault(user, {})
                self.test[user][movie] = int(rating)


    def UserSimilarity(self):
        #建立物品-用户的倒排表
        self.item_users = dict()
        for user,items in self.train.items():
            for i in items.keys():
                if i not in self.item_users:
                    self.item_users[i] = set()
                self.item_users[i].add(user)

        #计算用户-用户共现矩阵
        C = dict()  #用户-用户共现矩阵
        N = dict()  #用户产生行为的物品个数
        for i,users in self.item_users.items():
            for u in users:
                N.setdefault(u,0)
                N[u] += 1
                C.setdefault(u,{})
                for v in users:
                    if u == v:
                        continue
                    C[u].setdefault(v,0)
                    C[u][v] += 1

        #计算用户-用户相似度，余弦相似度
        self.W = dict()      #相似度矩阵
        for u,related_users in C.items():
            self.W.setdefault(u,{})
            for v,cuv in related_users.items():
                self.W[u][v] = cuv / math.sqrt(N[u] * N[v])
        return self.W, C, N

    #给用户user推荐，前K个相关用户
    def Recommend(self,u,K=3,N=10):
        rank = dict()
        action_item = self.train[u].keys()     #用户user产生过行为的item
        # v: 用户v
        # wuv：用户u和用户v的相似度
        for v,wuv in sorted(self.W[u].items(),key=lambda x:x[1],reverse=True)[0:K]:
            #遍历前K个与user最相关的用户
            # i：用户v有过行为的物品i
            # rvi：用户v对物品i的打分
            for i,rvi in self.train[v].items():
                if i in action_item:
                    continue
                rank.setdefault(i,0)
                # 用户对物品的感兴趣程度：用户u和用户v的相似度*用户v对物品i的打分
                rank[i] += wuv * rvi
        return dict(sorted(rank.items(),key=lambda x:x[1],reverse=True)[0:N])   #推荐结果的取前N个
    
    # 计算召回率和准确率
    def recallAndPrecision(self,k=8,nitem=10):
        hit = 0
        recall = 0
        precision = 0
        for user, items in self.test.items():
            rank = self.Recommend(user, K=k, N=nitem)
            hit += len(set(rank.keys()) & set(items.keys()))
            recall += len(items)
            precision += nitem
        return (hit / (recall * 1.0),hit / (precision * 1.0))

In [2]:
def print_2_dim_dic(dic, n=3):
    n = 0
    for u,v_cnt in dic.items():
        if n >= 3:
            break
        n += 1    
        m = 1
        for v, cnt in v_cnt.items():
            if m >= 3:
                break
            m += 1
            print(u, v, cnt)

def print_1_dim_dic(dic, n=3):
    n = 0
    for u,i_cnt in dic.items():
        if n >= 3:
            break
        n += 1    
        print(u, i_cnt)

def sort_2_dim_dic(dic, k, n=5):
    return sorted(dic[k].items(), key=lambda x:x[1],reverse=True)[:n]

def sort_1_dim_dic(dic, n=5):
    return sorted(dic.items(), key=lambda x:x[1],reverse=True)[:n]

def trans_dic_2_matrix(dic):
     return DataFrame(dic).T.fillna(0)

In [3]:
# user, movie, rating, _
path = os.path.join('ml-1m', 'ratings.dat')
ucf = UserBasedCF(path)

1,1193,5,978300760
1,661,3,978302109
1,914,3,978301968
1,3408,4,978300275
1,2355,5,978824291
1,1197,3,978302268
1,1287,5,978302039
1,2804,5,978300719
1,594,4,978302268
1,919,4,978301368
1,595,5,978824268


In [18]:
W,C,N = ucf.UserSimilarity()

In [19]:
# 用户共现矩阵
df_c = trans_dic_2_matrix(C)

In [20]:
df_c.shape

(6040, 6040)

In [21]:
df_c.iloc[:10,:10]

,1,10,100,1000,1001,1002,1003,1004,1005,1006
4360,13.0,47.0,4.0,7.0,19.0,2.0,5.0,68.0,6.0,5.0
2148,12.0,62.0,14.0,17.0,24.0,8.0,5.0,76.0,10.0,10.0
4107,9.0,62.0,19.0,20.0,22.0,5.0,8.0,90.0,17.0,6.0
2023,13.0,69.0,11.0,19.0,38.0,12.0,13.0,62.0,19.0,15.0
596,2.0,7.0,2.0,4.0,10.0,2.0,2.0,7.0,3.0,4.0
223,10.0,58.0,23.0,14.0,33.0,7.0,5.0,72.0,15.0,8.0
937,10.0,51.0,22.0,20.0,37.0,10.0,8.0,79.0,21.0,12.0
5543,16.0,76.0,14.0,18.0,27.0,9.0,7.0,91.0,17.0,6.0
504,1.0,14.0,2.0,3.0,7.0,2.0,3.0,12.0,4.0,1.0
3311,14.0,113.0,34.0,20.0,53.0,14.0,7.0,163.0,16.0,13.0


In [22]:
sort_2_dim_dic(C, '1')

[('1941', 29), ('2796', 27), ('1150', 27), ('4277', 26), ('5605', 26)]

In [23]:
# 用户产生行为的物品个数
print_1_dim_dic(N)

4360 181
2148 182
4107 246


In [24]:
sort_1_dim_dic(N)

[('4169', 1617), ('1680', 1274), ('4277', 1205), ('1941', 1110), ('889', 1075)]

In [25]:
# 用户和用户相似度矩阵
df_w = trans_dic_2_matrix(W)

In [26]:
df_w.shape

(6040, 6040)

In [27]:
df_w.iloc[:10, :10]

,1,10,100,1000,1001,1002,1003,1004,1005,1006
4360,0.158856,0.205499,0.040090,0.070805,0.087084,0.022161,0.079235,0.272912,0.056639,0.060289
2148,0.146233,0.270338,0.139930,0.171481,0.109698,0.088399,0.079017,0.304180,0.094139,0.120247
4107,0.094335,0.232528,0.163345,0.173526,0.086492,0.047522,0.108745,0.309833,0.137653,0.062057
2023,0.150372,0.285578,0.104360,0.181920,0.164866,0.125863,0.195010,0.235542,0.169778,0.171208
596,0.060030,0.075178,0.049237,0.099381,0.112580,0.054433,0.077850,0.069007,0.069561,0.118470
223,0.123222,0.255723,0.232454,0.142798,0.152520,0.078214,0.079900,0.291391,0.142786,0.097272
937,0.102549,0.187135,0.185044,0.169772,0.142317,0.092988,0.106393,0.266081,0.166363,0.121429
5543,0.169438,0.287976,0.121601,0.157786,0.107245,0.086423,0.096134,0.316509,0.139074,0.062698
504,0.017728,0.088803,0.029080,0.044023,0.046545,0.032150,0.068970,0.069869,0.054779,0.017493
3311,0.090345,0.260920,0.179960,0.106834,0.128285,0.081922,0.058582,0.345476,0.079763,0.082781


In [28]:
sort_2_dim_dic(W, '520')

[('3410', 0.3023480825550552),
 ('4425', 0.30024961868137817),
 ('5333', 0.2924767587528605),
 ('5689', 0.2855653829459588),
 ('5643', 0.2819657509649696)]

In [29]:
recomend = ucf.Recommend('520')
recomend

{'260': 4.182895541193608,
 '296': 4.182895541193608,
 '2858': 4.175122681265091,
 '541': 3.890418782440748,
 '1333': 3.59016916375937,
 '2321': 3.5802978399571757,
 '1580': 3.5802978399571757,
 '1961': 3.5802978399571757,
 '2739': 3.5781993760834983,
 '1210': 3.5781993760834983}

In [30]:
ucf.recallAndPrecision()

(0.06857348852268592, 0.3401324503311258)

## 用户相似度的改进

余弦计算相似度
<div align=center><font size=5 width="50%" height="50%">$W_{uv}=\frac{|N(u)\cap N(v)|}{\sqrt|N(u)||N(v)|}$</font></div>

以图书为例，两个用户都曾经购买过《新华字典》，这丝毫不能说明他们兴趣相似，因为绝大部分中国人小时候都买过《新华字典》，但如果两个用户都买过《数据挖掘导论》，那可以认为他们的兴趣相似，因为只有研究过数据挖掘的人才会买这本书。

换句话说，两个用户对冷门物品采取同样的行为，更能说明他们兴趣的相似度。因此，John S.Breese 在论文中提出了如下公式，根据用户行为计算用户兴趣相似度：

<div align=center><font size=5 width="50%" height="50%">$W_{uv}=\frac{\sum_{i \in N(u) \cap N(v) \frac{1}{log1+|N(i)|}}}{\sqrt|N(u)||N(v)|}$</font></div>

该公式通过 <font size=5 width="50%" height="50%">$\frac{1}{log1+|N(i)|}$</font> 惩罚了用户u和用户v共同兴趣列表中热门物品对他们相似度的影响。𝑁(𝑖) 是对物品𝑖有过行为的用户集合

In [17]:
def UserSimilarity(self):
    #建立物品-用户的倒排表
    self.item_users = dict()
    for user,items in self.train.items():
        for i in items.keys():
            if i not in self.item_users:
                self.item_users[i] = set()
            self.item_users[i].add(user)

    #计算用户-用户共现矩阵
    C = dict()  #用户-用户共现矩阵
    N = dict()  #用户产生行为的物品个数
    for i,users in self.item_users.items():
        for u in users:
            N.setdefault(u,0)
            N[u] += 1
            C.setdefault(u,{})
            for v in users:
                if u == v:
                    continue
                C[u].setdefault(v,0)
                C[u][v] += 1 / math.log(1+len(u))

    #计算用户-用户相似度，余弦相似度
    self.W = dict()      #相似度矩阵
    for u,related_users in C.items():
        self.W.setdefault(u,{})
        for v,cuv in related_users.items():
            self.W[u][v] = cuv / math.sqrt(N[u] * N[v])
    return self.W, C, N

# 物品协同
给用户推荐那些和他们之前喜欢的物品相似的物品。比如：该算法会因为你购买过《数据挖掘导论》而给你推荐《机器学习》。

ItemCF并不是利用物品内容属性计算物品相似度，而是通过分析用户的行为记录计算物品之间的相似度。

该算法认为物品A和物品B之间具有很大的相似度是因为用喜欢物品A的用户大都喜欢物品B。

## 目标
1. 计算物品之间的相似度。
2. 根据物品的相似度和用户的历史行为给用户生成推荐列表。

## 物品相似度
可以理解为：喜欢物品i的用户中有多少比例的用户也喜欢物品j
<div align=center><font size=5 width="50%" height="50%">$W_{ij}=\frac{|N(i) \cap N(j)|}{|N(i)|}$</font></div>

$|N(i) \cap N(j)|$是同时喜欢物品i和物品j的用户数，$N(i)$是喜欢物品i的用户数

<font size=4>**思考：如果物品j很热门，很多人都喜欢，那么$w_{ij}$就会很大，接近1。**</font>

因此，该公式会造成任何物品都会和热门物品有很大的相似度，这对于致力于挖掘长尾信息的推荐系统来说显然不是一个好的特性。

为避免推荐出热门的物品，可以用下面的公式：

<div align=center><font size=5 width="50%" height="50%">$W_{ij}=\frac{|N(i) \cap N(j)|}{\sqrt|N(i)||N(j)|}$</font></div>

这个公式惩罚了物品j的权重，因此减轻了热门物品会和很多物品相似的可能性。

和UserCF算法类似，ItemCF算法计算物品相似度时，也可以建立用户-物品倒排表(对每个用户建立一个包含他喜欢的物品列表。

```python
def ItemSimilarity(self):
        #建立物品-物品的共现矩阵
        C = dict()  #物品-物品的共现矩阵
        N = dict()  #物品被多少个不同用户购买
        for user,items in self.train.items():
            for i in items.keys():
                N.setdefault(i,0)
                N[i] += 1
                C.setdefault(i,{})
                for j in items.keys():
                    if i == j: 
                        continue
                    C[i].setdefault(j,0)
                    C[i][j] += 1
        #计算相似度矩阵
        self.W = dict()
        for i,related_items in C.items():
            self.W.setdefault(i,{})
            for j,cij in related_items.items():
                self.W[i][j] = cij / (math.sqrt(N[i] * N[j]))
        return self.W, C, N
```

最左边是输入的用户行为记录，每一行代表一个用户感兴趣的物品集合。然后，对于每个物品集合，将里面的物品两两加1，得到一个矩阵。最终将这些矩阵相加得到共现矩阵C，其中C[i][j]记录了同时喜欢物品i和物品j的用户数，最后，将C矩阵归一化得到物品之间的余弦相似度矩阵W。
![image.png](物品-物品共现矩阵.png)

## 给用户推荐和他喜欢的物品列表中最相似的物品
计算用户u对物品j感兴趣程度：
<div align=center><font size=5 width="50%" height="50%">$P_{uj}=\sum_{i \in N(u) \cap S(j,K)}w_{ji}r_{ui}$</font></div>

$N(u)$是用户喜欢的物品的集合，$S(j,K)$是和物品j最相似的K个物品的集合，$w_{ji}$是物品j和物品i的相似度，$r_{ui}$是用户u对物品i的兴趣。

![image.png](用户对物品j的感兴趣程度.png)

## 物品协同代码实现

In [47]:
class ItemBasedCF:
    def __init__(self, path):
        self.train = {} #用户-物品的评分表 训练集
        self.test = {} #用户-物品的评分表 测试集
        self.generate_dataset(path)

    def loadfile(self, path):
        with open(path, 'r', encoding='utf-8') as fp:
            for i, line in enumerate(fp):
                yield line.strip('\r\n')

    
    def generate_dataset(self, path, pivot=0.7):
        #读取文件，并生成用户-物品的评分表和测试集
        i = 0
        for line in self.loadfile(path):
            user, movie, rating, _ = line.split('::')
            if i <= 10:
                print('{},{},{},{}'.format(user, movie, rating, _))
            i += 1
            if random.random() < pivot:
                self.train.setdefault(user, {})
                self.train[user][movie] = int(rating)
            else:
                self.test.setdefault(user, {})
                self.test[user][movie] = int(rating)


    def ItemSimilarity(self):
        #建立物品-物品的共现矩阵
        C = dict()  #物品-物品的共现矩阵
        N = dict()  #物品被多少个不同用户购买
        for user,items in self.train.items():
            for i in items.keys():
                N.setdefault(i,0)
                N[i] += 1
                C.setdefault(i,{})
                for j in items.keys():
                    if i == j: 
                        continue
                    C[i].setdefault(j,0)
                    C[i][j] += 1
        #计算相似度矩阵
        self.W = dict()
        for i,related_items in C.items():
            self.W.setdefault(i,{})
            for j,cij in related_items.items():
                self.W[i][j] = cij / (math.sqrt(N[i] * N[j]))
        return self.W, C, N

    #给用户user推荐，前K个相关用户
    def Recommend(self,u,K=3,N=10):
        rank = dict()
        action_item = self.train[u]     #用户u产生过行为的item和评分
        for i,score in action_item.items():
            # j：物品j
            # wj：物品i和物品j的相似度
            for j,wj in sorted(self.W[i].items(),key=lambda x:x[1],reverse=True)[0:K]:                
                if j in action_item.keys():
                    continue
                rank.setdefault(j,0)
                # 用户u对物品j感兴趣程度：用户对物品i的打分 * 物品i和物品j的相似度
                rank[j] += score * wj
        return dict(sorted(rank.items(),key=lambda x:x[1],reverse=True)[0:N])
    
    # 计算召回率和准确率
    # 召回率 = 推荐的物品数 / 所有物品集合
    # 准确率 = 推荐对的数量 / 推荐总数
    def recallAndPrecision(self,k=8,nitem=10):
        hit = 0
        recall = 0
        precision = 0
        for user, items in self.test.items():
            rank = self.Recommend(user, K=k, N=nitem)
            hit += len(set(rank.keys()) & set(items.keys()))
            recall += len(items)
            precision += nitem
        return (hit / (recall * 1.0),hit / (precision * 1.0))

In [48]:
path = os.path.join('ml-1m', 'ratings.dat')
icf = ItemBasedCF(path)

1,1193,5,978300760
1,661,3,978302109
1,914,3,978301968
1,3408,4,978300275
1,2355,5,978824291
1,1197,3,978302268
1,1287,5,978302039
1,2804,5,978300719
1,594,4,978302268
1,919,4,978301368
1,595,5,978824268


In [49]:
# 计算物品
i_W, i_C, i_N = icf.ItemSimilarity()

In [50]:
# 物品-物品共现矩阵
df_ic = trans_dic_2_matrix(i_C)

In [51]:
df_ic.iloc[:10,:10]

,661,3408,1197,1287,594,919,595,2398,2918,1035
1193,101.0,226.0,415.0,182.0,177.0,427.0,209.0,122.0,303.0,191.0
661,0.0,77.0,173.0,53.0,130.0,153.0,161.0,46.0,104.0,124.0
3408,77.0,0.0,275.0,94.0,110.0,219.0,148.0,61.0,202.0,161.0
1197,173.0,275.0,0.0,234.0,275.0,540.0,336.0,129.0,539.0,260.0
1287,53.0,94.0,234.0,0.0,120.0,222.0,139.0,83.0,138.0,112.0
594,130.0,110.0,275.0,120.0,0.0,312.0,271.0,101.0,203.0,189.0
919,153.0,219.0,540.0,222.0,312.0,0.0,326.0,143.0,358.0,285.0
595,161.0,148.0,336.0,139.0,271.0,326.0,0.0,93.0,233.0,190.0
2398,46.0,61.0,129.0,83.0,101.0,143.0,93.0,0.0,111.0,95.0
2918,104.0,202.0,539.0,138.0,203.0,358.0,233.0,111.0,0.0,199.0


In [52]:
df_ic.shape

(3662, 3662)

In [53]:
# 物品被多少个不同用户购买
print_1_dim_dic(i_N)

1193 1196
661 370
3408 895


In [54]:
# 物品和物品相似度矩阵
df_iw = trans_dic_2_matrix(i_W)

In [55]:
df_iw.shape

(3662, 3662)

In [56]:
df_iw.iloc[:10, :10]

,661,3408,1197,1287,594,919,595,2398,2918,1035
1193,0.151829,0.218440,0.292423,0.238719,0.220044,0.353784,0.220968,0.217116,0.275414,0.224168
661,0.000000,0.133807,0.219167,0.124985,0.290565,0.227912,0.306037,0.147182,0.169958,0.261653
3408,0.133807,0.000000,0.224001,0.142527,0.158082,0.209753,0.180884,0.125492,0.212251,0.218434
1197,0.219167,0.224001,0.000000,0.258658,0.288113,0.377050,0.299376,0.193471,0.412883,0.257163
1287,0.124985,0.142527,0.258658,0.000000,0.234026,0.288543,0.230540,0.231717,0.196775,0.206208
594,0.290565,0.158082,0.288113,0.234026,0.000000,0.384355,0.426010,0.267252,0.274351,0.329814
919,0.227912,0.209753,0.377050,0.288543,0.384355,0.000000,0.341541,0.252180,0.322455,0.331457
595,0.306037,0.180884,0.299376,0.230540,0.426010,0.341541,0.000000,0.209281,0.267803,0.281974
2398,0.147182,0.125492,0.193471,0.231717,0.267252,0.252180,0.209281,0.000000,0.214749,0.237316
2918,0.169958,0.212251,0.412883,0.196775,0.274351,0.322455,0.267803,0.214749,0.000000,0.253903


In [57]:
recomend = icf.Recommend('520')

In [58]:
recomend

{'357': 12.211174702319198,
 '1196': 11.311378182752293,
 '594': 10.112069204301593,
 '1997': 9.126319045056363,
 '2997': 8.89347728210522,
 '2424': 8.728050947383766,
 '1350': 8.557122538227246,
 '1079': 7.909900119597647,
 '904': 7.521405968310045,
 '2145': 7.476983443692152}

In [59]:
icf.recallAndPrecision()

(0.07688866483940214, 0.38130794701986753)

## 物品相似度的归一化
ItemCF相似度矩阵按最大值归一化，可以提高推荐准确率。

<div align=center><font size=5 width="50%" height="50%">$w_{ij}=\frac{w_{ij}}{max_{j}w_{ij}}$</font></div>

案例：电影网站中，ItemCF计算出来的相似度一般是纪录片和纪录片的相似度更近，动画片和动画片的相似度更近，而动画片和纪录片的相似度却不一定相同。

假设物品有纪录片和动画片两类，纪录片之间的相似度为0.5，动画片之间的相似度为0.6，而纪录片和动画片之间的相似度是0.2。

在这种情况下，如果用户喜欢了5个纪录片和5个动画片，用ItemCF进行推荐，推荐的就都是动画片，因为动画片之间相似度大。

但如果归一化之后，纪录片之间的相似度就变成了1，动画片的相似度有变成了1，在这种情况下，用户如果喜欢了5个纪录片和5个动画片，那么他的推荐列表中，纪录片和动画片的数目也应该是大致相等的。

一般来说，热门的类的物品中相似度一般已经大，用户对热门物品的行为也比较多，如果不进行归一化，就会推荐比较热门的类里面的物品。

![image.png](ItemCF和ItemCF-Norm对比.png)

# 工业界推荐系统中常见的问题

## 工业界，协同召回的流程
1. 数据处理
    - 对行为少的不活跃的用户进行过滤，行为少的用户，数据太过于稀疏，召回难度大
    - 对用户中热门物品进行过滤，热门物品可能大部分用户都有过行为
    - 非常活跃的用户，用户协同可能会出现一种情况，就是每个用户的topN相似用户里都有这些非常活跃的用户，所以需要适当过滤掉这些用
2. 建立用户embedding和物品embedding，或者可以像案例这样，直接建立共现矩阵，也可以训练embedding
3. 计算用户和N个用户的相似度，保存N个相似用户曾经看过的TopK个物品
4. 模型(矩阵)进行定期更新（1、这个根据不同项目组的情况，可能是一天更新一次，也可能不是，看具体的情况； 2、更新的时候使用前N天(N一般可以为3-10天)的较活跃用户的数据进行更新）
5. 每次召回一次N条，刷完N条就再继续召回
    - 还有可能用户两次行为(上拉/下滑)之间间隔很长时间，也会进行重新召回
    - 每次召回的数量，需要根据召回通道数以及各个召回通道配置的召回占比进行配置
6. 为了保证用户不疲劳，一般情况下，利用user-cf计算召回结果后，会做一定的类别的去重，保证召回覆盖度。
7. 实际过程中，根据公司核心用户的数据量大小，考虑实现工具，若数据量较大，可使用spark进行用户协同的结果计算
8. 如果用户量实在太过巨大，可考虑使用稀疏存储的方式进行存储，即只存储含有1(或者其他值)的位置坐标索引index以及对应的值

## 用户行为大多是隐性的

用户的行为大部分都不会直接表现出来，以新闻类网站为例，用户阅读一篇文章，都不能很明确的表现出用户喜欢这类物品，有时候需要综合用户的点击，曝光，收藏，分享，阅读时长，评论等行为，做一个加权，然后再归一化，最终计算出一个合理的值来表示用户的喜好，往往这类行为不容易收集，而且也是比较稀疏的，需要积累很长时间，才能积累到足够的数据

## 用户协同和物品协同的使用场景

UserCF给用户推荐那些和他兴趣相投的用户喜欢的物品，而ItemCF是给用户推荐那些和他之前喜欢的物品类似的物品。

从算法原理来看，UserCF推荐结果着重于反映和用户兴趣相似的小群体的热点，而ItemCF的推荐结果着重于维系用户的历史兴趣。UserCF的推荐更社会化，反映了用户所在的小型兴趣群体中物品的热门程度。ItemCF的推荐更加个性化，反映了用户自己的兴趣传承。

### UserCF使用场景

**新闻网站**
1. 用户的兴趣不是很细化，绝大多数用户都喜欢看热点新闻，个性化推荐需求不强，比如有些用户喜欢看体育新闻，有些喜欢看社会新闻，特别细粒度的兴趣一般不存在。所以，新闻推荐，更强调抓住新闻热点，热门程度和时效性是个性化新闻推荐的重点，个性化需求相对比这两点略次要。

2. 技术角度来看，新闻类物品更新非常快，每时每刻都有新内容出现，而ItemCF需要维护一张物品-物品矩阵，有物品更新，这个表也需要更新，更新比较频繁，从技术和存储上来说，都比较困难。绝大多数物品相关度表都只能做到一天更新一次，在新闻领域这是不可接受的。而UserCF只需要维护用户相似性表，虽然用户对于新用户也需要更新相似度表，但在新闻网站中，物品更新速度远快于新用户加入速度。而且对于新用户，可以给他推热门的新闻，或者用冷启动策略去实现。

### ItemCF使用场景

**图书，电子商务，电影网站**
1. 这些网站中，用户的兴趣往往比较固定和持久的。比如，一个技术人员可能都是在购买技术方面的书籍，而他对书的热门程度并不是那么敏感。
2. 这些网站中个性化推荐的任务就是帮用户发现和他研究领域相关的物品。而且这些网站的物品更新不会特别快，一天更新一次物品相似度矩阵对他们来说不会造成太大的损失，用户是可以接受的。

## 协同过滤的改进
![image.png](协同过滤的改进.png)
### 基础算法

图1为最简单的计算物品相关度的公式，分子为同时喜好 item-i & item-j 的用户数。
	
### 对热门物品惩罚

但是图1存在一个问题，如果 item-j 是很热门的商品，导致很多喜欢 item-i 的用户都喜欢 item-j，这时 wij 就会非常大。同样，几乎所有的物品都和 item-j 的相关度非常高，这显然是不合理的。

图2中分母通过引入 N(j) 来对 item-j 的热度进行惩罚。

### 对热门物品进一步惩罚

如果 item-j 极度热门，上面的算法还是不够的。举个例子，《Harry Potter》非常火，买任何一本书的人都会购买它，即使通过图2的方法对它进行了惩罚，但是《Harry Potter》仍然会获得很高的相似度。这就是推荐系统领域著名的 Harry Potter Problem。

图1的方式，即便引入了N(j)，因为j非常热门，所以$|N(i) \cap N(j)|$就会越来越接近N(i)。即便上面的公式已经考虑到了j的流行度，但在实际应用中，热门的j仍然会获得比较大的相似度。

如果需要进一步对热门物品惩罚，可以继续修改公式为如图3所示，在分母上加大对热门物品的惩罚。通过调节参数 α ($α \in [0.5, 1]$)，α 越大，惩罚力度越大，热门物品的相似度越低，整体结果的平均热门程度越低。

![image.png](惩罚流行度后的ItemCF推荐效果对比.png)

对比不同的α惩罚热门物品后，ItemCF算法的推荐性能。

当α=0.5时，就是标准的ItemCF算法，从离线实验结果来看，α只有在取值为0.5时，才会导致最高的准确率和召回率，而物品α<0.5还是α>0.5都不会带来这两个指标的提高。但是看覆盖率和平均流行度就可以发现，α越大，覆盖率就越高，并且结果的平均热门程度会降低。

因此，通过这种方式可以适当牺牲准确率和召回率，显著提升结果的覆盖率和新颖性（降低流行度提高新颖性）。

### 对活跃用户惩罚

同样的，Item-based CF 也需要考虑活跃用户（即一个活跃用户（专门做刷单）可能买了非常多的物品）的影响，活跃用户对物品相似度的贡献应该小于不活跃用户。图4为集合了该权重的算法。

## 推荐系统中常用的评估指标
目前工业界还没有什么好的评估指标去衡量一个推荐系统。当然，在线下训练的时候，比如协同过滤，你可以像案例这样，把数据集分成训练集和测试集，数据需要做一些处理，比如，训练集是前面6天的数据，测试集是后面一天的数据，这么分的目的：不能在测试集看到用户在未来时间点发生的行为，避免时间泄露。可以线下大概评估一下协同效果。但是，用户兴趣往往是多变的，用户的行为可能这段时间喜欢娱乐，科技，可能后面一段时间喜欢财经，美女。所以最后，模型的效果，还是要根据线上用户的实际情况去评判，这也只是个参考值。

所以，现有的评估指标很难去衡量一个模型或者召回结果的好坏，只能通过看用户的行为数据，推荐的物品，用户点击的多，或者分享，评论，收藏等，通过数据表现，召回能力，曝光和点击数量，推荐物品覆盖度等一些指标，去判断召回的好坏。这些一般要根据公司实际业务，更侧重什么，去调整模型和算法。

比如更侧重曝光和点击，侧重用户体验，停留时长，召回多样性等。

## 协同过滤的优缺点
### 协同过滤优点

协同推荐是应用最广泛的推荐算法。因为基于内容推荐的算法，需要给物品打上标签，给用户建用户画像，才能实现匹配推荐。相比之下，协同过滤简单了许多。它是仅使用用户行为的进行推荐，我们不需要对物品或信息进行完整的标签化分析，避免了一些人可能难以量化描述的概念的标签构建，又可以很好地发现用户的潜在兴趣偏好。
    

### 协同过滤缺点

因为协同过滤依赖用户的历史数据，面对新的用户或者新的物品，在开始的时候没有数据或数据较少时，协同过滤算法无法做出推荐。需要等数据积累，或者其他方案进行弥补缺陷，也就是常说的冷启动的问题。

机器学习领域，当精确的方式不行难以计算或者速度太慢的时候，往往会选择牺牲一点精度，达到差不多但非常快速的效果。SVD就是其中的一个例子。

没有完美的算法，只有最合适的算法。现在的实践，也不是单纯用协同过滤来做推荐，而是将他们作为其中的一个或几个召回策略来使用。